In [2]:
import json
import cv2
from pathlib import Path
DATA_DIR = Path("../data")

In [2]:
def show_image_annotations(idx: int):
    img_path = DATA_DIR / f"img_{idx:04d}.jpg"
    json_path = DATA_DIR / f"img_{idx:04d}.json"

    annotations = json.loads(json_path.read_text())
    dart_boxes = annotations["dart_boxes"]
    markers = [annotations["D11_14"], annotations["D19_3"], annotations["D20_1"], annotations["D6_10"]]
    boxes = [*dart_boxes, *markers]

    img = cv2.imread(img_path)
    for cx, cy, w, h in boxes:
        topleft = (int(cx - w / 2), int(cy - h / 2))
        bottomright = (int(cx + w / 2), int(cy + h / 2))
        cv2.rectangle(img, topleft, bottomright, (0, 255, 255), 2)

    cv2.imshow(img_path.stem, img)
    cv2.waitKey(0)
    try:
        cv2.destroyWindow(img_path.stem)
    except cv2.error:
        pass


## Convert JSON to YOLO .txt Files

In [21]:
class_ids = {0: "D11_14", 1: "D19_3", 2: "D20_1", 3: "D6_10", 4: "dart_hit"}
class_ids_back = {"D11_14": 0, "D19_3": 1, "D20_1": 2, "D6_10": 3, "dart_hit": 4}


def to_yolo(json_path: Path, img_size: tuple[int, int] = (640, 640)):
    data = json.loads(json_path.read_text())
    dart_boxes = data["dart_boxes"]
    rows = ""
    for cx, cy, w, h in dart_boxes:
        cx /= img_size[0]
        cy /= img_size[1]
        w /= img_size[0]
        h /= img_size[1]
        rows += f"4 {cx:.6f} {cy:.6f} {w:.6f} {h:.6f}\n"

    for k, v in data.items():
        if k not in class_ids_back:
            continue
        cx, cy, w, h = v
        cx /= img_size[0]
        cy /= img_size[1]
        w /= img_size[0]
        h /= img_size[1]
        rows += f"{class_ids_back[k]} {cx:.6f} {cy:.6f} {w:.6f} {h:.6f}\n"
    
    out = json_path.with_suffix(".txt")
    out.write_text(rows)

for f in DATA_DIR.glob("*.json"):
    to_yolo(f)


## Organize dataset accroding to YOLO

<!-- ![](https://github.com/ultralytics/docs/releases/download/0/two-persons-tie-2.avif) -->

In [22]:
import numpy as np
import shutil
img_paths = list(DATA_DIR.glob("*.jpg"))
total_images = len(img_paths)
train_num, validation_num = 0.7 * total_images, 0.3 * total_images
np.random.shuffle(img_paths)

for i, img in enumerate(img_paths):
    label = img.with_suffix(".txt")
    if i < train_num:
        img_folder = DATA_DIR / "dart_positions/images/train"
        label_folder = DATA_DIR / "dart_positions/labels/train"
    else:
        img_folder = DATA_DIR / "dart_positions/images/val"
        label_folder = DATA_DIR / "dart_positions/labels/val"
        
    shutil.move(img, img_folder)
    shutil.move(label, label_folder)
    img.with_suffix(".json").unlink()
    # print(img, img_folder)
    # print(label, label_folder)
    

## Remove out of picture bounding boxes

In [ ]:
for f in DATA_DIR.rglob("*.txt"):
    contents = f.read_text()
    to_remove = []
    for i, line in enumerate(contents.splitlines()):
        id, cx, cy, w, h = line.split()
        cx, cy, w, h = map(float, (cx, cy, w, h))
        id = int(id)

        if any(x < 0 or x > 1 for x in [cx, cy, w, h]):
            to_remove.append(i)
            print(next(DATA_DIR.rglob(f"*{f.with_suffix('.jpg').name}")).resolve())
            print(f.resolve(), ":", line)

    if to_remove:
        new = ""
        for i, line in enumerate(contents.splitlines()):
            if i in to_remove:
                continue
            new += line + "\n"
        f.write_text(new)